# Distributed Representation for Word Embeddings 

In this programming assignment, we will experiment with distributed representations of words. We'll also see how such an embedding can be constructed by applying principal component analysis to a suitably transformed matrix of word co-occurrence probabilities. For computational reasons, we'll use the moderately sized **Brown corpus of present-day American English** for this.

## 1. Accessing the Brown corpus

The *Brown corpus* is available as part of the Python Natural Language Toolkit (`nltk`).

In [242]:
import numpy as np
import pickle
import nltk
nltk.download('brown')
nltk.download('stopwords')
from nltk.corpus import brown, stopwords
from scipy.cluster.vq import kmeans2
from sklearn.decomposition import PCA

[nltk_data] Downloading package brown to
[nltk_data]     /Users/benjaminaltermatt/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/benjaminaltermatt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The corpus consists of 500 samples of text drawn from a wide range of sources. When these are concatenated, they form a very long stream of over a million words, which is available as `brown.words()`. Let's look at the first 50 words.

In [243]:
for i in range(20):
    print (brown.words()[i],)

The
Fulton
County
Grand
Jury
said
Friday
an
investigation
of
Atlanta's
recent
primary
election
produced
``
no
evidence
''
that


Before doing anything else, let's remove stopwords and punctuation and make everything lowercase. The resulting sequence will be stored in `my_word_stream`.

In [244]:
my_stopwords = set(stopwords.words('english'))
word_stream = [str(w).lower() for w in brown.words() if w.lower() not in my_stopwords]
my_word_stream = [w for w in word_stream if (len(w) > 1 and w.isalnum())]

Here are the initial 20 words in `my_word_stream`.

In [245]:
my_word_stream[:20]

['fulton',
 'county',
 'grand',
 'jury',
 'said',
 'friday',
 'investigation',
 'recent',
 'primary',
 'election',
 'produced',
 'evidence',
 'irregularities',
 'took',
 'place',
 'jury',
 'said',
 'presentments',
 'city',
 'executive']

## 2. Computing co-occurrence probabilities

**Task P1**: Complete the following code to get a list of words and their counts. Report how many times does the word "evidence" and "investigation" appears in the corpus.

In [246]:
N = len(my_word_stream)
words = []
totals = {}

## STUDENT: Your code here
# words: a python list of unique words in the document my_word_stream as the vocabulary
# totals: a python dictionary, where each word is a key, and the corresponding value
#         is the number of times this word appears in the document my_word_stream

for word in my_word_stream:
    if word not in totals:
        totals[word] = 1
        words.append(word)
    else:
        totals[word] += 1

## STUDENT CODE ENDS

In [247]:
## STUDENT: Report how many times does the word "evidence" and "investigation" appears in the corpus.
print('Word "',words[10],'" appears ',totals[words[10]], ' times')
print('Word "',words[5],'" appears ',totals[words[5]], ' times')
print('Word "evidence" appears', totals['evidence'],'times')
print('Word "investigation" appears', totals['investigation'], 'times')

Word " produced " appears  90  times
Word " friday " appears  60  times
Word "evidence" appears 204 times
Word "investigation" appears 51 times


** Task P2**: Decide on the vocabulary. There are two potentially distinct vocabularies: the words for which we will obtain embeddings (`vocab_words`) and the words we will consider when looking at context information (`context_words`). We will take the former to be all words that occur at least 20 times, and the latter to be all words that occur at least 100 times. We will stick to these choices for this assignment, but feel free to play around with them and find something better.

How large are these two word lists? Note down these numbers.

In [248]:
## STUDENT: Your code here

# are the vocab words a subset of context words? I'm gonna assume they aren't? Is that what you mean by distinct vocabularies?
# TODO: ask for clarification

vocab_words = list() # a list of words whose occurances (totals) are > 19
context_words =  list() # a list of words whose occurances (totals) are > 99
for word in words:
    if totals[word] >= 100:
        context_words.append(word)
    if totals[word] >= 20:
        vocab_words.append(word)



## STUDENT CODE ENDS
print('Number of vocabulary words ',len(vocab_words), ';')
print('Number of context words ',len(context_words), ';')

Number of vocabulary words  4720 ;
Number of context words  918 ;


**Task P3**: Get co-occurrence counts. These are defined as follows, for a small constant `window_size=2`.

* Let `w0` be any word in `vocab_words` and `w` any word in `context_words`.
* Each time `w0` occurs in the corpus, look at the window of `window_size` words before and after it. If `w` appears in this window, we say it appears in the context of (this particular occurrence of) `w0`.
* Define `counts[w0][w]` as the total number of times `w` occurs in the context of `w0`.

Complete the function `get_counts`, which computes the `counts` array and returns it as a dictionary (of dictionaries). Find how many times the word "fact" appears in the context of ”evidence" with window_size=2.

In [249]:
def get_counts(window_size=2):
    ## Input:
    #  window_size: for each word w0, its context includes window_size words before and after it. 
    #  For instance, if window_size = 2, it means we look at w1 w2 w0 w3 w4, where  w1, w2, w3, w4 are 
    #  context woreds
    ## Output:
    #  counts: a python dictionary (of dictionaries) where counts[w0][w] indicate the number of times the word w appears 
    #  in the context of w0 (Note: counts[w0] is also a python dictionary)
    
    counts = {}
    for w0 in vocab_words:
        counts[w0] = {} 
        
    ## STUDENT: Your code here
    for word_ind in range(len(my_word_stream)):
        w0 = my_word_stream[word_ind]

        # check if vocab word
        if w0 in vocab_words:

            # check the windows
            for offset in range(-1 * window_size, window_size + 1):
                if offset != 0 and 0 <= word_ind + offset < len(my_word_stream):
                    # make sure our index is legal
                    w = my_word_stream[word_ind + offset]

                    if w in counts[w0].keys():
                        counts[w0][w] += 1
                    elif w in context_words:
                        counts[w0][w] = 1
    
    ## End of codes
    return counts

In [250]:
## STUDENT: Report how many times the word "fact" appears in the context of ”evidence".
counts = get_counts(window_size=2)
print(counts['evidence']['fact'])

4


Define `probs[w0][]` to be the distribution over the context of `w0`, that is:
* `probs[w0][w] = counts[w0][w] / (sum of all counts[w0][])`

**Task P4**: Finish the function `get_co_occurrence_dictionary` that computes `probs`. Find the probability that the word "fact" appears in the context of ”evidence".

In [251]:
def get_co_occurrence_dictionary(counts):
    ## Input:
    #  counts: a python dictionary (of dictionaries) where counts[w0][w] indicate the number of times the word w appears 
    #  in the context of w0 (Note: counts[w0] is also a python dictionary)
    ## Output:
    #  probs: a python dictionary (of dictionaries) where probs[w0][w] indicate the probability that word w appears 
    #  in the context of word w0
    
    probs = {}
    
    ## STUDENT: Your code here
    for w0 in counts.keys():
        sum_con = 0
        for w in counts[w0].keys():
            sum_con += counts[w0][w]
        
        probs[w0] = {}
        for w in counts[w0].keys():
            probs[w0][w] = counts[w0][w] / sum_con
    
    ## End of codes
    return probs

In [252]:
## STUDENT: Report how many times the word "fact" appears in the context of ”evidence".
probs = get_co_occurrence_dictionary(counts)
print(probs['evidence']['fact'])

0.010723860589812333


The final piece of information we need is the frequency of different context words. The function below, `get_context_word_distribution`, takes `counts` as input and returns (again, in dictionary form) the array:

* `context_frequency[w]` = sum of all `counts[][w]` / sum of all `counts[][]` 

In [253]:
def get_context_word_distribution(counts):
    counts_context = {}
    sum_context = 0
    context_frequency = {}
    for w in context_words:
        counts_context[w] = 0
    for w0 in counts.keys():
        for w in counts[w0].keys():
            counts_context[w] = counts_context[w] + counts[w0][w]
            sum_context = sum_context + counts[w0][w]
    for w in context_words:
        context_frequency[w] = float(counts_context[w])/float(sum_context)
    return context_frequency

## 3. The embedding

**Task P5**: Based on the various pieces of information above, we compute the **pointwise mutual information matrix**:
* `PMI[i,j] = MAX(0, log probs[ith vocab word][jth context word] - log context_frequency[jth context word])`

Complete the code to compute PMI for every word i and context word j. Report the output of the code.

In [254]:
print ("Computing counts and distributions")
counts = get_counts(2)
probs = get_co_occurrence_dictionary(counts)
context_frequency = get_context_word_distribution(counts)
#
print ("Computing pointwise mutual information")
n_vocab = len(vocab_words)
n_context = len(context_words)
pmi = np.zeros((n_vocab, n_context))
for i in range(0, n_vocab):
    w0 = vocab_words[i]
    for w in probs[w0].keys():
        j = context_words.index(w)
        ## STUDENT: Your code here
        pmi[i,j] = max(0, np.log(probs[w0][w])) - np.log(context_frequency[w])
        ## Student end of code

Computing counts and distributions
Computing pointwise mutual information


In [255]:
# STUDENT: report the following number
print(pmi[vocab_words.index('evidence'),context_words.index('fact')])

6.223953583900972


The embedding of any word can then be taken as the corresponding row of this matrix. However, to reduce the dimension, we will apply principal component analysis (PCA).

See this nice tutorial on PCA: https://www.youtube.com/watch?v=fkf4IBRSeEc

Now reduce the dimension of the PMI vectors using principal component analysis. Here we bring it down to 100 dimensions, and then normalize the vectors to unit length.

In [256]:
pca = PCA(n_components=100)
vecs = pca.fit_transform(pmi)
for i in range(0,n_vocab):
    vecs[i] = vecs[i]/np.linalg.norm(vecs[i])

It is useful to save this embedding so that it doesn't need to be computed every time.

In [257]:
fd = open("embedding.pickle", "wb")
pickle.dump(vocab_words, fd)
pickle.dump(context_words, fd)
pickle.dump(vecs, fd)
fd.close()

## 4. Experimenting with the embedding

We can get some insight into the embedding by looking at some intersting use cases.

** Task P6**: Implement the following function that finds the nearest neighbor of a given word in the embedded space. Note down the answers to the following queries. 

In [258]:
def word_NN(w,vecs,vocab_words,context_words):
    ## Input:
    #  w: word w
    #  vecs: the embedding of words, as computed above
    #  vocab_words: vocabulary words, as computed in Task P2
    #  context_words: context words, as computed in Task P2
    ## Output:
    #  the nearest neighbor (word) to word w
    if not(w in vocab_words):
        print ("Unknown word")
        return
    
    ## Student: your code here
    w_vector = vecs[vocab_words.index(w)]
    closest_ind = 0
    min_dist = np.linalg.norm(w_vector - vecs[closest_ind])
    for i in range(len(vocab_words)):
        cur_dist = np.linalg.norm(w_vector - vecs[i])
        if min_dist > cur_dist and cur_dist > 0.0: # i think it should always be greater than 0 but okay
            closest_ind = i
            min_dist = cur_dist
    
    return vocab_words[closest_ind]
    ## Student: code ends

In [259]:
word_NN('world',vecs,vocab_words,context_words)

'one'

In [260]:
word_NN('learning',vecs,vocab_words,context_words)

'economy'

In [261]:
word_NN('technology',vecs,vocab_words,context_words)

'tribute'

In [262]:
word_NN('winter',vecs,vocab_words,context_words)

'afternoon'

** Task P7**: Implement the function that aims to solve the analogy problem:
A is to B as C is to ?
For example, A=King, B=Queen, C=man, and the answer for ? should be ideally woman (you will see that this may not be  the case using the distributed representation).

Finds the K-nearest neighbor of a given word in the embedded space. Note: instead of outputing only the nearest neighbor, you should find the K=10 nearest neighbors and see whether there is one in the list that makes sense. You should also exclude the words C in the output list.

Also report another set A, B, C and the corresponding answer output by your problem. See if it makes sense to you.

In [263]:
def find_analogy(A,B,C,vecs,vocab_words,context_words):
    ## Input:
    #  A, B, C: words A, B, C
    #  vecs: the embedding of words, as computed above
    #  vocab_words: vocabulary words, as computed in Task P2
    #  context_words: context words, as computed in Task P2
    ## Output:
    #  the word that solves the analogy problem
    ## STUDENT: Your code here
    
    # first, let's make sure all of our words exist
    if A not in vocab_words:
        print('{:s} is not in the vocab words.'.format(A))
    if B not in vocab_words:
        print('{:s} is not in the vocab words.'.format(B))
    if C not in vocab_words:
        print('{:s} is not in the vocab words.'.format(C))

    # get the indices
    a_ind = vocab_words.index(A)
    b_ind = vocab_words.index(B)
    c_ind = vocab_words.index(C)

    best_ind = 0
    min_dif = np.linalg.norm(vecs[best_ind] + vecs[a_ind] - vecs[b_ind] - vecs[c_ind])
    for w_ind in range(len(vocab_words)):
        cur_dif = np.linalg.norm(vecs[w_ind] + vecs[a_ind] - vecs[b_ind] - vecs[c_ind])
        if cur_dif < min_dif and cur_dif >= 0.0 and c_ind != w_ind:
            best_ind = w_ind
            min_dif = cur_dif
    
    return vocab_words[best_ind]

    ## STUDENT: your code ends


In [264]:
find_analogy('king','queen','man',vecs,vocab_words,context_words)

'like'

In [265]:
find_analogy('soil','grass','sun',vecs,vocab_words,context_words)

'grass'

In [266]:
word_NN('woman',vecs,vocab_words,context_words)

'knew'